In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"raghav3253","key":"6d255e9f0a7e4e49a388f0b4b1199b55"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!cp kaggle.json ~/.kaggle/

In [8]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!ls

  0% 0.00/7.01k [00:00<?, ?B/s]
100% 7.01k/7.01k [00:00<00:00, 5.59MB/s]
 86% 79.0M/92.0M [00:00<00:00, 65.5MB/s]
100% 92.0M/92.0M [00:01<00:00, 90.4MB/s]
example_submission.csv	fer2013.tar.gz	kaggle.json  sample_data


In [9]:
!ls

example_submission.csv	fer2013.tar.gz	kaggle.json  sample_data


In [0]:
!mkdir dataset
import tarfile
tar = tarfile.open("./fer2013.tar.gz")
tar.extractall('./dataset')
tar.close()

In [11]:
%cd ./dataset/fer2013
!mv fer2013.csv /content/

/content/dataset/fer2013


In [12]:
%cd /content

/content


In [13]:
!ls

dataset			fer2013.csv	kaggle.json
example_submission.csv	fer2013.tar.gz	sample_data


In [0]:
!rm -r dataset
!rm example_submission.csv
!rm fer2013.tar.gz
!rm -r sample_data

In [15]:
!ls
!pwd

fer2013.csv  kaggle.json
/content


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import categorical_accuracy
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

In [0]:
img_width = 48
img_height = 48
dataset = pd.read_csv('./fer2013.csv')
label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names = ['emotion','pixels','usage']

In [0]:
X = dataset.iloc[:, 1].values
y = dataset.iloc[:, 0].values

In [0]:
image_data=np.empty([len(X),img_width,img_height,1],dtype=float)
i=0
for pixel_string in X:
  pixels = [float(pixel) for pixel in pixel_string.split(' ')]
  image = np.asarray(pixels).reshape(img_height, img_width)
  image_data[i,:,:,0]=image
  i=i+1

In [0]:
#number of images in dataset : 35887
#number of images for training 80% split : 28709
#number of images for validation  20 % split :  7178

num_class=len(label_map)
train=image_data[0:28709,:,:,:]
validation=image_data[28709:,:,:,:]
train_y=y[0:28709]
validation_y=y[28709:]
train_y = (np.arange(num_class) == train_y[:, None]).astype(np.float32)
validation_y = (np.arange(num_class) == validation_y[:, None]).astype(np.float32)

In [65]:
def my_model():
    model = Sequential()
    input_shape = (48,48,1)
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(0.35))
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    
    return model
  
model=my_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 48, 48, 64)        1664      
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 48, 48, 64)        102464    
_________________________________________________________________
batch_normalization_17 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 24, 24, 128)       204928    
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 24, 24, 128)       409728    
_________________________________________________________________
batch_normalization_18 (Batc (None, 24, 24, 128)       512       
__________

In [66]:
batch_size=64
epochs=30
verbose=1
history = model.fit(
    train, train_y,
    validation_data = (validation,validation_y),
    batch_size=batch_size,
    verbose=verbose,
    epochs=epochs
)

Train on 28709 samples, validate on 7178 samples
Epoch 1/30
28709/28709 [==============================] - 24s 825us/step - loss: 1.7090 - acc: 0.3336 - val_loss: 1.5288 - val_acc: 0.4067
Epoch 2/30
28709/28709 [==============================] - 22s 766us/step - loss: 1.3800 - acc: 0.4730 - val_loss: 1.2588 - val_acc: 0.5156
Epoch 3/30
28709/28709 [==============================] - 22s 755us/step - loss: 1.1859 - acc: 0.5497 - val_loss: 1.1945 - val_acc: 0.5464
Epoch 4/30
28709/28709 [==============================] - 22s 750us/step - loss: 1.0862 - acc: 0.5939 - val_loss: 1.1998 - val_acc: 0.5449
Epoch 5/30
28709/28709 [==============================] - 22s 757us/step - loss: 0.9883 - acc: 0.6267 - val_loss: 1.1002 - val_acc: 0.5855
Epoch 6/30
28709/28709 [==============================] - 22s 760us/step - loss: 0.9082 - acc: 0.6623 - val_loss: 1.0692 - val_acc: 0.6035
Epoch 7/30
28709/28709 [==============================] - 22s 755us/step - loss: 0.8198 - acc: 0.6940 - val_loss: 1.0

In [0]:
# Save entire model to a HDF5 file
model.save('my_model.h5')

In [0]:
train_datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

#generating data for train and validation set
train_data = train_datagen.flow(train,train_y, batch_size=32)
validation_data = validation_datagen.flow(validation,validation_y,batch_size=32)